## Golomb编码
对8bit图,MAXVAL=127,RANGE=128,bpp=8,qbpp=8,LIMIT=32

In [77]:
def bin8(num):
    outnum = bin(num)[2:]
    while(len(outnum)!=8):
        outnum = '0' + outnum
    return outnum

def one_element_coding(num):
    outnum = '0' * num + '1'
    return outnum

def golomb_coding(MErrval,k,LIMIT=32,qbpp=8):
    MErrval_R_k = MErrval>>k
    outstr = ''
    if MErrval_R_k < LIMIT-qbpp-1:
        outstr+=one_element_coding(MErrval_R_k)
        outstr+=bin8(MErrval)[-k:]
    else:
        outstr+=one_element_coding(LIMIT-qbpp-1)
        outstr+=bin8(MErrval-1)[-qbpp:]
    return outstr

def golomb_decoding(outstr,k,LIMIT=32,qbpp=8):
    pass

## 游程编码

## 总程序

初始化变量和变量组

In [78]:
# for 8 bits picture
MAXVAL=255
RANGE=256
bpp=8
qbpp=8
LIMIT=32
MAX_C=127
MIN_C=-128

# A，N从0到366，B，C从0到364，Nn从365到366，A初始化为全4，其余初始化都为全0
A = [4 for _ in range(366)]
B = [0 for _ in range(364)]
C = [0 for _ in range(364)]
N = [0 for _ in range(366)]
Nn= {365:0,366:0}

RUNcnt = 0
RUNindex = 0

J = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# pic(index starts from 1)
# 0     0       90      74
# 68    50      43      205
# 64    145     145     145
# 100   145     145     145

origin_pic = [[0,0,90,74],
[68,50,43,205],
[64,145,145,145],
[100,145,145,145]]

自定义函数

In [85]:
def reconstruction_pic(origin_pic):
    expand_pic = [[0 for _ in range(len(origin_pic[0])+2)] for _ in range(len(origin_pic)+1)]
    for i in range(len(origin_pic)):
        for j in range(len(origin_pic[0])):
            expand_pic[i+1][j+1]=origin_pic[i][j]
    
    for i in range(len(origin_pic)-1):
        expand_pic[i+2][0]=origin_pic[i][0]
        expand_pic[i+1][len(origin_pic[0])+1]=origin_pic[i][len(origin_pic[0])-1]
    return expand_pic

def get_pixels(expand_pic,hang_index,lie_index):
    _hang_index = hang_index+1
    _lie_index = lie_index+1

    x = expand_pic[_hang_index     ][_lie_index      ]
    a = expand_pic[_hang_index     ][_lie_index    - 1  ]
    b = expand_pic[_hang_index  -1 ][_lie_index    ]
    c = expand_pic[_hang_index - 1 ][_lie_index   -1 ]
    d = expand_pic[_hang_index -1  ][_lie_index  +1  ]
    return x,a,b,c,d

def get_gradient(a,b,c,d):
    D1 = d-b
    D2 = b-c
    D3 = c-a
    return D1,D2,D3

def mode_choose(D1,D2,D3):
    '''
    return mode = 0 : run mode
    return mode = 1 : regular mode
    '''
    if D1==0 and D2==0 and D3==0:
        mode = 0
    else:
        mode = 1
    return mode



regular mode functions

In [99]:
def quantinize(D):
    if D<=-21:
        Q=-4
    elif D<=-7:
        Q=-3
    elif D<=-3:
        Q=-2
    elif D<0:
        Q=-1
    elif D==0:
        Q=0
    elif D<=3:
        Q=1
    elif D<=7:
        Q=2
    elif D<=21:
        Q=3
    else:
        Q=4
    return Q

def mapping_and_sign(Q1,Q2,Q3):

    if Q1<0 or (Q1==0 and Q2<0) or (Q1==0 and Q2==0 and Q3<0):
        SIGN = -1
        Q1,Q2,Q3=-Q1,-Q2,-Q3
    else:
        SIGN = 1
    Q = 81*Q1+9*Q2+Q3
    return Q,SIGN

In [ ]:
if __name__ == "__main__":

    # for 8 bits picture
    MAXVAL=255
    RANGE=256
    bpp=8
    qbpp=8
    LIMIT=32
    MAX_C=127
    MIN_C=-128

    # A，N从0到366，B，C从0到364，Nn从365到366，A初始化为全4，其余初始化都为全0
    A = [4 for _ in range(366)]
    B = [0 for _ in range(364)]
    C = [0 for _ in range(364)]
    N = [0 for _ in range(366)]
    Nn= {365:0,366:0}

    RUNcnt = 0
    RUNindex = 0

    J = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]

    # pic(index starts from 1)
    # 0     0       90      74
    # 68    50      43      205
    # 64    145     145     145
    # 100   145     145     145

    origin_pic = [[0,0,90,74],
    [68,50,43,205],
    [64,145,145,145],
    [100,145,145,145]]

    expand_pic = reconstruction_pic(origin_pic)

    for i in range(4):
        for j in range(4):
            x,a,b,c,d = get_pixels(expand_pic, i,j)
            D1,D2,D3 = get_gradient(x,a,b,c,d)
            mode = mode_choose(D1,D2,D3)

            if mode==1:#regular mode
                
